In [ ]:
import geopandas
import shapely
import rioxarray
import pathlib
import rioxarray.merge
import matplotlib
import numpy

# Correct spurious negative values in the Wellington_2013 Waikanae DEM
There are some spurious negative values with ground (and vegetation) classifications in the Wellington 2013 datasets that are causing very negative pixel values in the Wellington_2013 derived Waikanae DEMs. Here we will focus on removing these in a post processing step. In future these will be detected and corrected in the DEM generation process. See [Spurious negative elevations](https://niwa.sharepoint.com/:w:/r/sites/FloodResilienceAotearoa/NationalFloodMapping/Shared%20Documents/DEM%20Generation/Spurious%20negative%20elevations.docx?d=w052158aedcab4951aa62d54c00ef2622&csf=1&web=1&e=WRQMaa) for documentation of the problem, and [issue 65](https://github.com/rosepearson/GeoFabrics/issues/65) for the planned correction.

In [ ]:
base_path = pathlib.Path("C:/Users/pearsonra/Documents/data/Waikanae/caches/Wellington_2013")

## Load in the dense DEM and detect spurious values
Plan:
1. Load in 4_5m_dense_dem.nc
2. Detect and remove the spurious negative pixels
3. Save the results

This can then be used as the new dense DEM in the production of new DEMs with bathymetry using GeoFabrics

In [ ]:
# laod in DEM
with rioxarray.rioxarray.open_rasterio(base_path / "4_5m_dense_dem.nc", masked=True) as dem:
    dem.load()
dem = dem.copy(deep=True)  # Deep copy is required to ensure the opened file is properly unlocked
dem.rio.set_crs(2193);

Create a copy of the DEM with 1 where values are less than -10, and 0 elsewhere. Keep the threshold at -10, as when -1 start getting values near the mouth. Checked to ensure all values with a filter of -10 as at the location of the spurious negative pixels inland of the Peka Paka road onramp to the Kapiti Expressway. 

In [ ]:
dem_mask = dem.copy(deep=True)
dem_mask.data[dem.data < -10] = 1
dem_mask.data[dem.data >= -10] = 0
dem_mask.data[numpy.isnan(dem.data)] = 0 # to stop values adjacent becoming nan during filtering
dem_mask.to_netcdf(base_path / "4_5m_dense_dem_masked.nc")

Create a mask and filter with a rectangular file to expand the region to be removed by one in each pixel direction

In [ ]:
import scipy.ndimage
dem_no_negative_mask=scipy.ndimage.convolve(dem_mask.data, numpy.ones([1, 3, 3]))

Save the filtered dem while ensuring positive indexing

In [ ]:
dem_no_negative = dem.copy(deep=True)
dem_no_negative.data[dem_no_negative_mask > 0] = numpy.nan

In [ ]:
# Ensure positive indexing
x = dem_no_negative.x
y = dem_no_negative.y
if x[0] > x[-1]:
    x = x[::-1]
if y[0] > y[-1]:
    y = y[::-1]
dem_no_negative = dem_no_negative.reindex(x=x, y=y)

In [ ]:
dem_no_negative.to_netcdf(base_path / "4_5m_dense_dem_no_negative.nc")